In [24]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [25]:
# import warnings filter to filter our sklearn future warnings
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [26]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [27]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [28]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [29]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d", \
                                                                    "price_change_percentage_14d", "price_change_percentage_30d", \
                                                                    "price_change_percentage_60d"]])

market_data_scaled[0:10]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287],
       [ 0.89187071,  1.32729453,  0.80021418, -0.0571479 ,  0.77865311],
       [ 0.01139724,  2.57225091,  1.10164693, -0.49049541, -0.93195402],
       [ 0.1025298 ,  1.50800146,  0.64888506,  0.32895924, -0.4863489 ],
       [ 0.07749718,  0.33429656,  0.85851986, -0.01264647, -0.36647699],
       [ 0.44895151, -0.19068428, -0.24804322,  0.05163401, -0.52966614]])

In [30]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(
    market_data_scaled,
    columns=["price_change_percentage_24h", "price_change_percentage_7d", \
             "price_change_percentage_14d", "price_change_percentage_30d", \
             "price_change_percentage_60d"])

# Copy the crypto names from the original data
scaled_df["coinid"] = df_market_data.index

# Set the coinid column as index
scaled_df = scaled_df.set_index("coinid")


# Display sample data
scaled_df.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d
coinid,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203


---

### Find the Best Value for k Using the Original Data.

In [31]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [32]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters=i, random_state=0)
# 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(scaled_df)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)

In [33]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [34]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
kmeans_elbow = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Crypto Elbow Curve", 
    xticks=k
)

kmeans_elbow

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3 is the best value for `k`. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [35]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=1)

In [36]:
# Fit the K-Means model using the scaled data
model.fit(scaled_df)

KMeans(n_clusters=3, random_state=1)

In [37]:
# Predict the clusters to group the cryptocurrencies using the scaled data
kmeans_predictions = model.predict(scaled_df)

# Print the resulting array of cluster values.
kmeans_predictions

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1],
      dtype=int32)

In [38]:
# Create a copy of the DataFrame
scaled_df_prediction = scaled_df.copy()

In [39]:
# Add a new column to the DataFrame with the predicted clusters
scaled_df_prediction['kmeans_clusters'] = kmeans_predictions

# Display sample data
scaled_df_prediction.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,kmeans_clusters
coinid,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,0


In [40]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
kmeans_plot = scaled_df_prediction.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="kmeans_clusters",
    hover_cols=["coinid"],
    use_index=True
)

kmeans_plot

:NdOverlay   [kmeans_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coinid)

---

### Optimize Clusters with Principal Component Analysis.

In [41]:
# Create a PCA model instance and set `n_components=3`.
pca=PCA(n_components=3)

In [42]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
scaled_pca = pca.fit_transform(scaled_df)
# View the first five rows of the DataFrame. 
scaled_pca[:5]

array([[ 0.79510406, -0.66290603, -0.15388459],
       [ 0.41376913, -1.04777288,  0.15762251],
       [-0.19550822,  0.51826138, -0.20153516],
       [-0.26074757,  0.34060069, -0.14574095],
       [ 1.96101883, -2.23972436, -0.18298129]])

In [43]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_variance = pca.explained_variance_ratio_

pca_variance

array([0.47862164, 0.26608254, 0.1684978 ])

In [44]:
#Total Variance
pca_variance.sum()*100

91.32019782433119

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 91.3% of the total variance is condensed into the 3 princial components.

In [45]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
pca_df = pd.DataFrame(
    scaled_pca,
    columns=["PC1", "PC2", "PC3"]
)
# Copy the crypto names from the original data
pca_df["coinid"] = df_market_data.index

# Set the coinid column as index
pca_df = pca_df.set_index("coinid")

# Display sample data

pca_df.head()

,PC1,PC2,PC3
coinid,,,
bitcoin,0.795104,-0.662906,-0.153885
ethereum,0.413769,-1.047773,0.157623
tether,-0.195508,0.518261,-0.201535
ripple,-0.260748,0.340601,-0.145741
bitcoin-cash,1.961019,-2.239724,-0.182981


---

### Find the Best Value for k Using the PCA Data

In [46]:
# Create a list with the number of k-values from 1 to 11
pca_k = list(range(1, 11))

In [47]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in pca_k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    pca_k_model = KMeans(n_clusters=i, random_state=0)
# 2. Fit the model to the data using `df_market_data_scaled`
    pca_k_model.fit(pca_df)
# 3. Append the model.inertia_ to the inertia list
    pca_inertia.append(pca_k_model.inertia_)

In [48]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": pca_k, "inertia": pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)

In [49]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow = df_pca_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="PCA Elbow Curve", 
    xticks=k
)

pca_elbow

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3 is the best value for `k`


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ fromt the best K value from the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [50]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=1)

In [51]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

KMeans(n_clusters=3, random_state=1)

In [52]:
# Predict the clusters to group the cryptocurrencies using the PCA data
kmeans_pca_predictions = model.predict(pca_df)

# Print the resulting array of cluster values.
kmeans_pca_predictions

array([0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2],
      dtype=int32)

In [53]:
# Create a copy of the DataFrame with the PCA data
pca_clusters = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_clusters["pca_clusters"] = kmeans_pca_predictions

# Display sample data
pca_clusters.head()

,PC1,PC2,PC3,pca_clusters
coinid,,,,
bitcoin,0.795104,-0.662906,-0.153885,0
ethereum,0.413769,-1.047773,0.157623,0
tether,-0.195508,0.518261,-0.201535,2
ripple,-0.260748,0.340601,-0.145741,2
bitcoin-cash,1.961019,-2.239724,-0.182981,0


In [54]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


pca_plot = pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="pca_clusters",
    hover_cols=["coinid"],
    use_index=True
)

pca_plot

:NdOverlay   [pca_clusters]
   :Scatter   [PC1]   (PC2,coinid)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [55]:
# Side-by-side Elbow curves

dual_elbow = pca_elbow + kmeans_elbow

dual_elbow



:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [56]:
# Composite plot to contrast the Elbow curves

composite_elbow = pca_elbow * kmeans_elbow

# Overwrite Title
composite_elbow.opts(title="Composite Elbow")

composite_elbow

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [57]:
#side-by-side scatter
dual_scatter = kmeans_plot + pca_plot

dual_scatter

:Layout
   .NdOverlay.I  :NdOverlay   [kmeans_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coinid)
   .NdOverlay.II :NdOverlay   [pca_clusters]
      :Scatter   [PC1]   (PC2,coinid)

In [58]:
# Composite plot to contrast the clusters
composite_scatter = kmeans_plot * pca_plot

# Overwrite Title
composite_scatter.opts(title="Composite Scatter")

composite_scatter

:Overlay
   .NdOverlay.I  :NdOverlay   [kmeans_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coinid)
   .NdOverlay.II :NdOverlay   [pca_clusters]
      :Scatter   [PC1]   (PC2,coinid)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 